# Positive Predictive Value Analysis for Imaging Modalities

## Introduction to PPV in Medical Imaging

Positive Predictive Value (PPV) is a key metric in medical imaging that tells us how reliable a positive test result is. It answers the question: "When the test says there's something there, how often is it actually there?"

PPV is calculated as:
```
PPV = (True Positives) / (True Positives + False Positives)
```

PPV works together with other metrics:
- Sensitivity tells us how good a test is at finding actual problems
- Specificity shows how good it is at ruling out problems that aren't there
- Modality concordance looks at when different imaging methods agree with each other

For parathyroid imaging, high PPV means we can trust positive findings to guide surgery. When multiple imaging methods agree (concordance), it often increases our confidence in the results.


In [ ]:
# Import needed libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Read the data
data = pd.read_excel("Data/Clean_imaging_data.xlsx")


## 1. SPECT/CT Analysis

SPECT/CT combines two imaging methods:
- SPECT (Single-Photon Emission Computed Tomography) shows how tissues function
- CT (Computed Tomography) provides detailed anatomical images
Together, they help locate parathyroid glands by showing both their activity and exact position.

Analysis Steps:
1. Filter data for SPECT/CT results
2. Count true positives (correctly identified glands) and false positives (incorrect identifications)
3. Calculate PPV to see how reliable positive SPECT/CT findings are
4. Create visualization of results


In [ ]:
# SPECT/CT PPV Analysis
# Code will go here to calculate SPECT/CT PPV


## 2. SPECT/CT + Sestamibi Concordance Analysis

Concordance means two different imaging methods agree on their findings. When SPECT/CT and Sestamibi both point to the same location, it might increase our confidence in the result. This analysis looks at whether concordance actually leads to better accuracy.

Analysis Steps:
1. Find cases where both SPECT/CT and Sestamibi tests were performed
2. Identify concordant results (when both tests point to the same location)
3. Calculate PPV for these concordant cases to see if agreement improves accuracy
4. Create visualization comparing concordant vs. single-modality results


In [ ]:
# SPECT/CT + Sestamibi Concordance PPV Analysis
# Code will go here to calculate concordance PPV


## 3. Sestamibi Analysis

Sestamibi is a nuclear medicine scan that uses a radioactive tracer to find overactive parathyroid glands. It's one of the most common first-line tests because it's widely available and relatively simple to perform. The tracer concentrates in areas with lots of mitochondria, which are abundant in parathyroid adenomas.

Analysis Steps:
1. Isolate Sestamibi scan results from our dataset
2. Count true positives (confirmed parathyroid tissue) and false positives (other tissue types)
3. Calculate PPV to determine how often positive Sestamibi findings are correct
4. Create visualization of Sestamibi performance


In [ ]:
# Sestamibi PPV Analysis
# Code will go here to calculate Sestamibi PPV


## 4. 4D CT Analysis

4D CT (Four-Dimensional Computed Tomography) adds the dimension of time to regular CT scanning. It captures images at multiple time points as contrast flows through the tissues, helping distinguish parathyroid glands from other structures based on how they absorb and release contrast. This newer technique might offer better anatomical detail than traditional methods.

Analysis Steps:
1. Extract all 4D CT results from our dataset
2. Identify true positives (actual parathyroid glands) and false positives (other tissues)
3. Calculate PPV to assess the reliability of positive 4D CT findings
4. Create visualization showing 4D CT performance metrics


In [ ]:
# 4D CT PPV Analysis
# Code will go here to calculate 4D CT PPV


In [ ]:
# Combined Analysis
# Code will go here to compare all modalities
# Will make final visualization comparing all PPVs


## 5. Combined Analysis of All Modalities

This final section brings together results from all imaging methods to understand their relative strengths. Each method has its own advantages - some are better at finding certain gland locations, while others might be more reliable overall. By comparing PPVs, we can help guide which tests might be most useful in different situations.

Analysis Steps:
1. Collect PPV results from all imaging methods (SPECT/CT, Sestamibi, 4D CT, and concordant findings)
2. Create a comparison table showing PPV, sample sizes, and confidence intervals
3. Make a bar chart to visually compare PPVs across methods
4. Perform statistical tests to see if some methods are significantly better than others
5. Write a summary of key findings and potential clinical implications

This analysis helps us understand:
- Which imaging method is most reliable when positive
- Whether using multiple tests together improves accuracy
- If newer methods (like 4D CT) perform better than traditional ones
- How to best combine different imaging methods for optimal results
